In [24]:
import pandas
import time
import numpy as np
import preprocessors
import estimators

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from estimators import LSHNearestNeighbors

In [25]:
path = "../data/query.yaHotelId.showInTop.sure.final.tsv"

In [26]:
df = pandas.read_csv(path, sep="\t")
print("Изначальная размерность данных:", df.shape,";", "Количество отелей:", len(df["yaHotelId"].unique()))

Изначальная размерность данных: (588232, 4) ; Количество отелей: 95884


In [34]:
sure_df = df[df["sure"]]
filtered_values = [value[0] for value in sure_df["yaHotelId"].value_counts().iteritems() if value[1] >= 5]
filtered_df = sure_df[sure_df["yaHotelId"].isin(filtered_values)]
print("Получившаяся размерность данных:", filtered_df.shape, ";", "Количество отелей:", len(filtered_df["yaHotelId"].unique()))

Получившаяся размерность данных: (92427, 4) ; Количество отелей: 7696


In [35]:
df_train, df_test = train_test_split(filtered_df, test_size=0.01)
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

In [36]:
df_train.head()

,query,yaHotelId,showIntop,sure
0,аполлония резорт,4283126,True,True
1,majestic colonial punta cana hotel,1004918,True,True
2,centara seaview resort khao lak 4 khao lak,1007612,True,True
3,brown atrium hotel rimini,1011768,True,True
4,alan xafira deluxe resort spa8 турция анталья ...,1003046,True,True


In [37]:
ids = set(df_train["yaHotelId"])
df_hotels = pandas.read_csv("../data/hotel_all.csv")
df_hotels = df_hotels[df_hotels["id"].isin(ids)]
df_hotels.rename(columns={"id":"yaHotelId", "all": "query"}, inplace=True)
df_hotels.drop(labels=["Unnamed: 0"], axis=1, inplace=True)
df_hotels.dropna(axis=0, how="any", inplace=True)
print(df_hotels.shape)
df_hotels = preprocessors.buildup_set(df_hotels)
print(df_hotels.shape)

(7072, 2)
(21216, 2)


In [38]:
df_tr = df_train.append(df_hotels)
df_tr.reset_index(drop=True, inplace=True)

In [39]:
df_tr.tail()

,query,showIntop,sure,yaHotelId
112713,Artemis Apartments,NaN,NaN,4702510
112714,Sunset Apartments,NaN,NaN,4707616
112715,Apartments Medena,NaN,NaN,4717444
112716,ibis Budapest Heroes Square,NaN,NaN,4718594
112717,Отель Оганес,NaN,NaN,4729441


In [40]:
prep = preprocessors.Preprocessor([preprocessors.num_filter_preprocess, preprocessors.mystem_preprocess, preprocessors.dictionary_preprocess])
vectorizer = TfidfVectorizer(preprocessor=prep.preprocess)
y_train = np.array(df_tr["yaHotelId"])
X_train = vectorizer.fit_transform(df_tr["query"])
X_train.shape

(112718, 12671)

In [41]:
y_test = np.array(df_test["yaHotelId"])
X_test = vectorizer.transform(df_test["query"])

In [42]:
clf = LSHNearestNeighbors(n_estimators=15, n_candidates=200, n_neighbors=9, mode="parzen_window", 
                          parzen_func=estimators.epan_parzen_function)
clf.fit(X_train, y_train)

new1


LSHNearestNeighbors(answers='one', mode='parzen_window', n_candidates=200,
          n_estimators=None, n_neighbors=9,
          parzen_func=<function epan_parzen_function at 0x7fea7295b510>)

In [43]:
y_pred = clf.predict(X_test)
print("accuracy:", accuracy_score(y_test, y_pred))

accuracy: 0.918918918919


In [44]:
print("precision:", precision_score(y_test, y_pred, average="weighted"))

precision: 0.924504504505


/home/alexander/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [45]:
print("recall:", recall_score(y_test, y_pred, average="weighted"))

recall: 0.918918918919


/home/alexander/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [46]:
clf.get_quantile(p=0.9)

0    0.066414
Name: 0.9, dtype: float64

In [47]:
prep.get_quantile(p=0.9)

0    0.00035
Name: 0.9, dtype: float64

In [48]:
k = 0
for i in range(len(y_test[:50])):
    if y_test[i] != y_pred[i]:
        k+=1
        print("[{}, {}]Right:{}, predicted:{}".format(k, i, y_test[i], y_pred[i]))
        print("text:{}".format(df_test["query"][i]))
        print("preprocessed text:{}".format(prep.preprocess(df_test["query"][i])))
        print("neighbors:", clf.get_neighbors(X_test[i,:]))
        print("--")
        print("another queries of right class:")
        for st in df[df["yaHotelId"] == y_test[i]]["query"].tolist()[:5]:
            print(st)
        print("--")
        print("another queries of predicted class:")
        for st in df[df["yaHotelId"] == y_pred[i]]["query"].tolist()[:5]:
            print(st)
        print("---------------------------------")

[1, 4]Right:1013207, predicted:2664200
text:гостиница ялта
preprocessed text:hotel yalta
neighbors: [2664200, 2664200, 1009854, 2664200, 2664200, 1013207, 1009854, 1013207, 1009854]
--
another queries of right class:
ялта интурист букинг ком
интурист ялта официальный сайт цены на 2015 год
ялта интурист цены
zknf bynehbcn
ялта интурист официальный сайт цены
--
another queries of predicted class:
отель таврида ялта официальный сайт
отель таврида ялта
таврида ялта
таврида ялта отель
таврида ялта официальный сайт
---------------------------------
[2, 20]Right:1003970, predicted:1011522
text:колония санта мария 4
preprocessed text:colony santa maria 4
neighbors: [1011522, 1011522, 1009722, 1011522, 1011522, 1011522, 1011522, 1011522, 1011356]
--
another queries of right class:
colonia santa maria
colonia santa maria 3
colonia santa maria 4
colonia santa maria 2
индия отель санта
--
another queries of predicted class:
hotel santa maria 3 искья
santa maria искья
santa maria 3 искья
hotel sant

1)Из данных по ошибкам видно, что иногда объекты правильного класса находятся ближе всего к нашему, но их меньше, поэтому происходит ошибочная классификация. 
2)Также можно видеть, что в запроса есть много составных слов, которые, вроде как, нужно использовать по отдельности или вообще использовать только пвторую часть слова. 
3)Вьетнам слишком часто копирует названия.

In [40]:
clf.predict(X_test[33, :])

/home/alexander/PycharmProjects/hotels/estimators.py:107: RuntimeWarning: invalid value encountered in double_scalars
  rating[k_n_d[i][0]] = self.parzen_func(k_n_d[i][1]/k_n_d[self.n_neighbors - 1][1])
/home/alexander/PycharmProjects/hotels/estimators.py:109: RuntimeWarning: invalid value encountered in double_scalars
  rating[k_n_d[i][0]] += self.parzen_func(k_n_d[i][1]/k_n_d[self.n_neighbors - 1][1])


[-1]

In [36]:
X_test[33]

<1x17261 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>